# Exploring CLIP

This notebook is based on--and the first third of it is almost identical to--"Interacting with CLIP.ipynb." I'm not sure who authored that notebook, but it is very similar to code snippets provided by OpenAI [in their documentation of the CLIP model.](https://github.com/openai/CLIP)

This is a self-contained notebook that shows how to download and run CLIP models, calculate the similarity between arbitrary image and text inputs, and perform zero-shot image classifications.

# Preparation for Colab

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will install the `clip` package and its dependencies, and check if PyTorch 1.7.1 or later is installed.

In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-dmqvj2n1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-dmqvj2n1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=809479b11b22095eb44f4371a050bb172aa510099421bf64751404c87bef3b56
  Stored in directory: /tmp/pip-ephem-wheel-cache-cauwa7b6/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import numpy as np
import torch

print("Torch version:", torch.__version__)

assert torch.__version__.split(".") >= ["1", "7", "1"], "PyTorch 1.7.1 or later is required"

Torch version: 2.5.1+cu121


# Loading the model

`clip.available_models()` will list the names of available CLIP models.

In [3]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 73.0MiB/s]


Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


# Image Preprocessing

We resize the input images and center-crop them to conform with the image resolution that the model expects. Before doing so, we will normalize the pixel intensity using the dataset mean and standard deviation.

The second return value from `clip.load()` contains a torchvision `Transform` that performs this preprocessing.



In [5]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7ada17eb39a0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

# Text Preprocessing

We use a case-insensitive tokenizer, which can be invoked using `clip.tokenize()`. By default, the outputs are padded to become 77 tokens long, which is what the CLIP models expects.

In [6]:
clip.tokenize("Hello World!")

tensor([[49406,  3306,  1002,   256, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)

# Setting up input images and texts

We are going to feed 8 example images and their textual descriptions to the model, and compare the similarity between the corresponding features.

The tokenizer is case-insensitive, and we can freely give any suitable textual descriptions.

In [7]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# images in skimage to use and their textual descriptions
descriptions = {
    "Luggage Template": "A person in denim dragging a Suitcase with Luggage Covers",
    "Luggage Non": "A person in denim dragging a Suitcase with Luggage Covers",
    "Rug Template": "Living room with parquet Area Rug and Sofa",
    "Rug Non": "Living room with parquet Area Rug and Sofa",
}

In [8]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))

for filename in [filename for filename in os.listdir(skimage.data_dir) if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg")]:
    name = os.path.splitext(filename)[0]
    if name not in descriptions:
        continue
    else:
        image = Image.open(os.path.join(skimage.data_dir, filename)).convert("RGB")

    plt.subplot(2, 4, len(images) + 1)
    plt.imshow(image)
    plt.title(f"{filename}\n{descriptions[name]}")
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))
    texts.append(descriptions[name])

plt.tight_layout()



FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/scikit-image/0.24.0/data'

<Figure size 1600x500 with 0 Axes>

## Building features

We normalize the images, tokenize each text input, and run the forward pass of the model to get the image and text features.

In [ ]:
image_input = torch.tensor(np.stack(images)).cuda()
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).cuda()

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

## Calculating cosine similarity

We normalize the features and calculate the dot product of each pair.

A few notes here. Functions like ```.cuda()``` and ```.cpu()``` are being used to put these matrices into a format where they can run on a Graphics Processing Unit (GPU), and then back into the normal format we expect in Python, which runs on a CPU.

The at-sign operator computes the dot product of two matrices, and the ```.T``` transposes a matrix.

In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

In [ ]:
count = len(descriptions)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])
for i, image in enumerate(original_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)

# First experiment

That diagonal line seems very neat and good. But how fragile is CLIP?

Are there plausible ways you could describe these images that *wouldn't* work as well? Go back to the cell where the images were associated with descriptions and see if you can make up alternate descriptions of these pictures that are just as accurate but don't use exactly the same terms or details.

When you use those descriptions for the cosine similarity matrix, how well does the diagonal hold up?

# Zero-Shot Image Classification

The strategy adopted here is to load a dataset with 100 categories and turn the category names into description strings on the pattern "This is a photo of a {label}." Then we use those strings to categorize the eight images from the experiment above.

Since the categories in CIFAR100 are not guaranteed to line up with the photos above, this approach may not always generate a good match! But it's a fair model of the results you might get if you tried to sort random images into a set of predefined categories.

In [ ]:
from torchvision.datasets import CIFAR100

cifar100 = CIFAR100(os.path.expanduser("~/.cache"), transform=preprocess, download=True)

In [ ]:
text_descriptions = [f"This is a photo of a {label}" for label in cifar100.classes]
text_tokens = clip.tokenize(text_descriptions).cuda()

**Turning cosine similarity into class probabilities:**

The math that follows is slightly interesting. We calculate cosine similarities for all the classes, as we did before.

But then we turn those numbers into probabilities using a *softmax* function. Essentially this replaces all the elements k1, k2, k3 of a vector with $e^{k1}, e^{k2}$ etc, and then normalizes the whole vector by dividing by sum(vector). This guarantees that all the class probabilities will add up to 1.

In [ ]:
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

In [ ]:
plt.figure(figsize=(16, 16))

for i, image in enumerate(original_images):
    plt.subplot(4, 4, 2 * i + 1)
    plt.imshow(image)
    plt.axis("off")

    plt.subplot(4, 4, 2 * i + 2)
    y = np.arange(top_probs.shape[-1])
    plt.grid()
    plt.barh(y, top_probs[i])
    plt.gca().invert_yaxis()
    plt.gca().set_axisbelow(True)
    plt.yticks(y, [cifar100.classes[index] for index in top_labels[i].numpy()])
    plt.xlabel("probability")

plt.subplots_adjust(wspace=0.5)
plt.show()

Hmm. confusing a cat with a sweet pepper, and text with a bed.

# What if we tried to categorize CIFAR images using the CIFAR labels?

Not surprisingly, this performs a little better.

The CIFAR images have been loaded as ordinary numpy arrays, but the "preprocess" function that crops them and sizes them appropriately for CLIP expects PIL "Image" objects, so we convert them into "Images" before preprocessing.


In [ ]:
image_input = torch.tensor(np.stack([preprocess(Image.fromarray(x)) for x in cifar100.data[0:8]])).cuda()


To generate textual descriptions of the images, we rely on the fact that cifar100 has a list of "targets" (integers that indicate the class of the corresponding image). These can be converted into text using the list of classes that is also in cifar100.

In [ ]:
print('Classes in order:', cifar100.classes[0:10])
print('Indices for the first eight pictures:', cifar100.targets[0:8])

text_tokens = clip.tokenize([f"This is a photo of a {cifar100.classes[label]}" for label in cifar100.targets[0:8]]).cuda()

The next part is entirely unchanged from above. We simply run the model to turn images and texts into vectors.

Then we compute cosine similarity by normalizing the vectors and taking their dot products.

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
similarity

This is largely unchanged from above, except that I've substituted cifar_descriptions and cifar images.

In [ ]:
count = 8
cifar_descriptions = [cifar100.classes[x] for x in cifar100.targets[0:8]]

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), cifar_descriptions, fontsize=18)
plt.xticks([])
for i, image in enumerate(cifar100.data[0:8]):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)

### Glance forward at homework

The section we just worked through will come back in Homework 7, due October 31. I'll ask you to use CLIP to classify several hundred images from CIFAR100, and then assess the accuracy of the model.

If you wanted to save this to your Google Drive you could uncomment the lines below.

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Second experiment, with movie posters

Let's see whether CLIP can recognize the cultural meaning in complex images: movie posters.

Your first mission here is to flesh out the code for measuring cosine similarity (using the models above).

Then, after you've measured the similarity of the movie posters to the seven descriptions I provide, I'll ask you to confer in small groups. Change aspects of the experiment so you can better understand why CLIP succeeds (where it succeeds) and why it fails when it fails. In particular, the posters for Carol, Love Story, and Text Matrix create interesting puzzles.

To run this section you'll need to upload ```posters.zip,``` by clicking the icon like a page with an up arrow on the far left. Then you can unzip it by running the cell below.

In [ ]:
!unzip posters.zip

In [ ]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))

for filename in [filename for filename in os.listdir('posters/') if filename.endswith(".jpg") and 'Text' not in filename]:
    # There are nine files in the folder and it's hard to display more than eight, so I've excluded one
    # with a "not in" condition. Later we can change which one is excluded by replacing 'Text' with
    # '2050'.

    image = Image.open(os.path.join('posters/', filename)).convert("RGB")

    plt.subplot(2, 4, len(images) + 1)
    plt.imshow(image)
    plt.title(f"{filename}")
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))
    texts.append(filename.replace('.jpg', ''))

plt.tight_layout()

In [ ]:
img_descriptions = ['a poster for an action movie',
                    'a poster for a romance movie',
                    'a poster for a bengali movie',
                    'a poster for star wars',
                    'a poster for the matrix',
                    'a poster for a science fiction movie',
                    'a love story']

In [ ]:
image_input = ?    # what would you need to put here?
text_tokens = ?    # and here?

In [ ]:
# what happens now?

In [ ]:
# and now?

In [ ]:
# and finally, how do you visualize the results as a similarity matrix?

# Third experiment

Create your own folder of 4-5 images (perhaps works of art from different periods or regions?) Zip it and upload it.

Then write your own text strings to see if CLIP can distinguish regions or periods from each other.